In [1]:
PROJECT_ID = "formazione-riccardo-zanella" #@param {type: "string"}
REGION = 'us-central1' #@param {type: "string"}
BUCKET_NAME = "bbs-2021-opml4b-explainability" #@param {type:"string"}

In [17]:
TRAIN_FILE = 'gs://'+BUCKET_NAME+'/data/train.csv'
TEST_FILE = 'gs://'+BUCKET_NAME+'/data/test.csv'
EXPORT_PATH = 'gs://' + BUCKET_NAME + '/explanations/mymodel'

## Build, train, and evaluate your model with Keras

This section shows how to build, train, evaluate, and get local predictions from a model by using the Keras [Sequential API](https://www.tensorflow.org/guide/keras/sequential_model). The model will takes your 10 features as input and predict the trip duration in minutes.

In [2]:
import tensorflow as tf
import pandas as pd

2021-10-22 14:33:49.308334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-22 14:33:49.308374: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
with tf.io.gfile.GFile(TRAIN_FILE) as f:
    train_data = pd.read_csv(f)

with tf.io.gfile.GFile(TEST_FILE) as f:
    test_data = pd.read_csv(f)

In [8]:
from typing import Tuple

def split_features_labels(dframe: pd.DataFrame, label_col: str = 'duration') -> Tuple[pd.DataFrame]:
    labels = dframe[label_col]
    features = dframe.drop(columns=[label_col])
    return features, labels

In [9]:
train_data, train_labels = split_features_labels(train_data)
test_data, test_labels = split_features_labels(test_data)

In [10]:
# Build your model
model = tf.keras.Sequential(name="bike_predict")
model.add(tf.keras.layers.Dense(64, input_dim=len(train_data.iloc[0]), activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))

# Compile the model and see a summary
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss='mean_squared_logarithmic_error', optimizer=optimizer)
model.summary()

2021-10-22 14:38:56.351778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-22 14:38:56.351815: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-22 14:38:56.351831: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (INJ-NB-126): /proc/driver/nvidia/version does not exist
2021-10-22 14:38:56.351988: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Create an input data pipeline with tf.data

In [12]:
batch_size = 256
epochs = 3

input_train = tf.data.Dataset.from_tensor_slices(train_data)
output_train = tf.data.Dataset.from_tensor_slices(train_labels)
input_train = input_train.batch(batch_size).repeat()
output_train = output_train.batch(batch_size).repeat()
train_dataset = tf.data.Dataset.zip((input_train, output_train))

### Train the model

In [13]:
# This will take about a minute to run
# To keep training time short, you're not using the full dataset
train_size = len(train_data)
model.fit(train_dataset, steps_per_epoch=train_size // batch_size, epochs=epochs)

Epoch 1/3
  69/2958 [..............................] - ETA: 4s - loss: 2.0662

2021-10-22 14:39:58.528991: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2958/2958 [==============================] - 5s 1ms/step - loss: 0.7057
Epoch 2/3
2958/2958 [==============================] - 4s 1ms/step - loss: 0.3599
Epoch 3/3
2958/2958 [==============================] - 4s 1ms/step - loss: 0.3608


In [19]:
model.save(EXPORT_PATH)
print(EXPORT_PATH)

2021-10-22 14:41:16.906603: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://bbs-2021-opml4b-explainability/explanations/mymodel/assets
gs://bbs-2021-opml4b-explainability/explanations/mymodel


### Evaluate the trained model locally

In [14]:
# Run evaluation
results = model.evaluate(test_data, test_labels)
print(results)

5917/5917 [==============================] - 4s 625us/step - loss: 0.3549
0.3549324870109558


In [15]:
# Send test instances to model for prediction
predict = model.predict(test_data[:5])

In [16]:
# Preview predictions on the first 5 examples from your test dataset
for i, val in enumerate(predict):
    print('Predicted duration: {}'.format(round(val[0])))
    print('Actual duration: {} \n'.format(test_labels.iloc[i]))

Predicted duration: 18
Actual duration: 22.0 

Predicted duration: 27
Actual duration: 31.0 

Predicted duration: 27
Actual duration: 23.0 

Predicted duration: 17
Actual duration: 14.0 

Predicted duration: 12
Actual duration: 9.0 

